## J.Jill Market Analysis

The goal of this analysis is to identify pockets of opportunity for the J.Jill brand by 

In [3]:
import csv
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

### Part 1: Data Pre - Processing

In [4]:
dat_orig = pd.read_csv('zip_codes_and_income.csv', low_memory = False)

The primary data set used for the base of this analysis was taken from the US Census within the [American Fact Finder.](https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_17_5YR_B19001&prodType=table) The data set was compiled by adding all 5-Digit ZIP Code Tabulation Areas within United States and Puerto Rico. From the website the data was downloaded into a csv file where it could be loaded as a pandas dataframe.

In [5]:
pd.options.display.max_columns = 300

In [6]:
dat_orig.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17,HD01_VD18,HD02_VD18,HD01_VD19,HD02_VD19,HD01_VD20,HD02_VD20,HD01_VD21,HD02_VD21,HD01_VD22,HD02_VD22,HD01_VD23,HD02_VD23,HD01_VD24,HD02_VD24,HD01_VD25,HD02_VD25,HD01_VD26,HD02_VD26,HD01_VD27,HD02_VD27,HD01_VD28,HD02_VD28,HD01_VD29,HD02_VD29,HD01_VD30,HD02_VD30,HD01_VD31,HD02_VD31,HD01_VD32,HD02_VD32,HD01_VD33,HD02_VD33,HD01_VD34,HD02_VD34,HD01_VD35,HD02_VD35,HD01_VD36,HD02_VD36,HD01_VD37,HD02_VD37,HD01_VD38,HD02_VD38,HD01_VD39,HD02_VD39,HD01_VD40,HD02_VD40,HD01_VD41,HD02_VD41,HD01_VD42,HD02_VD42,HD01_VD43,HD02_VD43,HD01_VD44,HD02_VD44,HD01_VD45,HD02_VD45,HD01_VD46,HD02_VD46,HD01_VD47,HD02_VD47,HD01_VD48,HD02_VD48,HD01_VD49,HD02_VD49,HD01_VD50,HD02_VD50,HD01_VD51,HD02_VD51,HD01_VD52,HD02_VD52,HD01_VD53,HD02_VD53,HD01_VD54,HD02_VD54,HD01_VD55,HD02_VD55,HD01_VD56,HD02_VD56,HD01_VD57,HD02_VD57,HD01_VD58,HD02_VD58,HD01_VD59,HD02_VD59,HD01_VD60,HD02_VD60,HD01_VD61,HD02_VD61,HD01_VD62,HD02_VD62,HD01_VD63,HD02_VD63,HD01_VD64,HD02_VD64,HD01_VD65,HD02_VD65,HD01_VD66,HD02_VD66,HD01_VD67,HD02_VD67,HD01_VD68,HD02_VD68,HD01_VD69,HD02_VD69
0,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; Householder under 25 years:,Margin of Error; Householder under 25 years:,Estimate; Householder under 25 years: - Less t...,Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $10,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $15,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $20,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $25,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $30,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $35,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $40,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $45,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $50,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $60,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $75,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $100,0...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $125,0...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $150,0...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $200,0...",Margin of Error; Householder under 25 years: -...,Estimate; Householder 25 to 44 years:,Margin of Error; Householder 25 to 44 years:,Estimate; Householder 25 to 44 years: - Less t...,Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $10,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $15,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $20,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $25,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $30,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $35,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Househo

In [7]:
dat_orig.shape

(33121, 141)

This data set was taken from us census data and can be found [here](https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_17_5YR_B19037&prodType=table). Filter for age of householder, household earning, and 5 digit zips.

The first step is to trim the dataset to only include the estimate columns (drop the margin of errors.)

In [8]:
dat_trim = dat_orig[['GEO.id2', 'HD01_VD01','HD01_VD02','HD01_VD03','HD01_VD04','HD01_VD05','HD01_VD06','HD01_VD07','HD01_VD08','HD01_VD09','HD01_VD10','HD01_VD11','HD01_VD12','HD01_VD13','HD01_VD14','HD01_VD15','HD01_VD16','HD01_VD17','HD01_VD18','HD01_VD19','HD01_VD20','HD01_VD21','HD01_VD22','HD01_VD23','HD01_VD24','HD01_VD25','HD01_VD26','HD01_VD27','HD01_VD28','HD01_VD29','HD01_VD30','HD01_VD31','HD01_VD32','HD01_VD33','HD01_VD34','HD01_VD35','HD01_VD36','HD01_VD37','HD01_VD38','HD01_VD39','HD01_VD40','HD01_VD41','HD01_VD42','HD01_VD43','HD01_VD44','HD01_VD45','HD01_VD46','HD01_VD47','HD01_VD48','HD01_VD49','HD01_VD50','HD01_VD51','HD01_VD52','HD01_VD53','HD01_VD54','HD01_VD55','HD01_VD56','HD01_VD57','HD01_VD58','HD01_VD59','HD01_VD60','HD01_VD61','HD01_VD62','HD01_VD63','HD01_VD64','HD01_VD65','HD01_VD66','HD01_VD67','HD01_VD68','HD01_VD69']]
dat_trim.head()

,GEO.id2,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,HD01_VD09,HD01_VD10,HD01_VD11,HD01_VD12,HD01_VD13,HD01_VD14,HD01_VD15,HD01_VD16,HD01_VD17,HD01_VD18,HD01_VD19,HD01_VD20,HD01_VD21,HD01_VD22,HD01_VD23,HD01_VD24,HD01_VD25,HD01_VD26,HD01_VD27,HD01_VD28,HD01_VD29,HD01_VD30,HD01_VD31,HD01_VD32,HD01_VD33,HD01_VD34,HD01_VD35,HD01_VD36,HD01_VD37,HD01_VD38,HD01_VD39,HD01_VD40,HD01_VD41,HD01_VD42,HD01_VD43,HD01_VD44,HD01_VD45,HD01_VD46,HD01_VD47,HD01_VD48,HD01_VD49,HD01_VD50,HD01_VD51,HD01_VD52,HD01_VD53,HD01_VD54,HD01_VD55,HD01_VD56,HD01_VD57,HD01_VD58,HD01_VD59,HD01_VD60,HD01_VD61,HD01_VD62,HD01_VD63,HD01_VD64,HD01_VD65,HD01_VD66,HD01_VD67,HD01_VD68,HD01_VD69
0,Id2,Estimate; Total:,Estimate; Householder under 25 years:,Estimate; Householder under 25 years: - Less t...,"Estimate; Householder under 25 years: - $10,00...","Estimate; Householder under 25 years: - $15,00...","Estimate; Householder under 25 years: - $20,00...","Estimate; Householder under 25 years: - $25,00...","Estimate; Householder under 25 years: - $30,00...","Estimate; Householder under 25 years: - $35,00...","Estimate; Householder under 25 years: - $40,00...","Estimate; Householder under 25 years: - $45,00...","Estimate; Householder under 25 years: - $50,00...","Estimate; Householder under 25 years: - $60,00...","Estimate; Householder under 25 years: - $75,00...","Estimate; Householder under 25 years: - $100,0...","Estimate; Householder under 25 years: - $125,0...","Estimate; Householder under 25 years: - $150,0...","Estimate; Householder under 25 years: - $200,0...",Estimate; Householder 25 to 44 years:,Estimate; Householder 25 to 44 years: - Less t...,"Estimate; Householder 25 to 44 years: - $10,00...","Estimate; Householder 25 to 44 years: - $15,00...","Estimate; Householder 25 to 44 years: - $20,00...","Estimate; Householder 25 to 44 years: - $25,00...","Estimate; Householder 25 to 44 years: - $30,00...","Estimate; Householder 25 to 44 years: - $35,00...","Estimate; Householder 25 to 44 years: - $40,00...","Estimate; Householder 25 to 44 years: - $45,00...","Estimate; Householder 25 to 44 years: - $50,00...","Estimate; Householder 25 to 44 years: - $60,00...","Estimate; Householder 25 to 44 years: - $75,00...","Estimate; Householder 25 to 44 years: - $100,0...","Estimate; Householder 25 to 44 years: - $125,0...","Estimate; Householder 25 to 44 years: - $150,0...","Estimate; Householder 25 to 44 years: - $200,0...",Estimate; Householder 45 to 64 years:,Estimate; Householder 45 to 64 years: - Less t...,"Estimate; Householder 45 to 64 years: - $10,00...","Estimate; Householder 45 to 64 years: - $15,00...","Estimate; Householder 45 to 64 years: - $20,00...","Estimate; Householder 45 to 64 years: - $25,00...","Estimate; Householder 45 to 64 years: - $30,00...","Estimate; Householder 45 to 64 years: - $35,00...","Estimate; Householder 45 to 64 years: - $40,00...","Estimate; Householder 45 to 64 years: - $45,00...","Estimate; Householder 45 to 64 years: - $50,00...","Estimate; Householder 45 to 64 years: - $60,00...","Estimate; Householder 45 to 64 years: - $75,00...","Estimate; Householder 45 to 64 years: - $100,0...","Estimate; Householder 45 to 64 years: - $125,0...","Estimate; Householder 45 to 64 years: - $150,0...","Estimate; Householder 45 to 64 years: - $200,0...",Estimate; Householder 65 years and over:,Estimate; Householder 65 years and over: - Les...,Estimate; Householder 65 years and over: - $10...,Estimate; Householder 65 years and over: - $15...,Estimate; Householder 65 years and over: - $20...,Estimate; Householder 65 years and over: - $25...,Estimate; Householder 65 years and over: - $30...,Estimate; Householder 65 years and over: - $35...,Estimate; Householder 65 years and over: - $40...,Estimate; Householder 65 years and over: - $45...,Estimate; Householder 65 years and over: - $50...,Estimate; Householder 65 years and over: - $60...,Estimate; Householder 65 years and over: - $75...,Estimate; Householder 65 years an

Column names need fixing. We'll make the first row (row 0) be our new columns names.

In [9]:
dat_trim.columns = dat_trim.iloc[0]
dat_trim = dat_trim.reindex(dat_trim.index.drop(0))
dat_trim.head()

,Id2,Estimate; Total:,Estimate; Householder under 25 years:,"Estimate; Householder under 25 years: - Less than $10,000","Estimate; Householder under 25 years: - $10,000 to $14,999","Estimate; Householder under 25 years: - $15,000 to $19,999","Estimate; Householder under 25 years: - $20,000 to $24,999","Estimate; Householder under 25 years: - $25,000 to $29,999","Estimate; Householder under 25 years: - $30,000 to $34,999","Estimate; Householder under 25 years: - $35,000 to $39,999","Estimate; Householder under 25 years: - $40,000 to $44,999","Estimate; Householder under 25 years: - $45,000 to $49,999","Estimate; Householder under 25 years: - $50,000 to $59,999","Estimate; Householder under 25 years: - $60,000 to $74,999","Estimate; Householder under 25 years: - $75,000 to $99,999","Estimate; Householder under 25 years: - $100,000 to $124,999","Estimate; Householder under 25 years: - $125,000 to $149,999","Estimate; Householder under 25 years: - $150,000 to $199,999","Estimate; Householder under 25 years: - $200,000 or more",Estimate; Householder 25 to 44 years:,"Estimate; Householder 25 to 44 years: - Less than $10,000","Estimate; Householder 25 to 44 years: - $10,000 to $14,999","Estimate; Householder 25 to 44 years: - $15,000 to $19,999","Estimate; Householder 25 to 44 years: - $20,000 to $24,999","Estimate; Householder 25 to 44 years: - $25,000 to $29,999","Estimate; Householder 25 to 44 years: - $30,000 to $34,999","Estimate; Householder 25 to 44 years: - $35,000 to $39,999","Estimate; Householder 25 to 44 years: - $40,000 to $44,999","Estimate; Householder 25 to 44 years: - $45,000 to $49,999","Estimate; Householder 25 to 44 years: - $50,000 to $59,999","Estimate; Householder 25 to 44 years: - $60,000 to $74,999","Estimate; Householder 25 to 44 years: - $75,000 to $99,999","Estimate; Householder 25 to 44 years: - $100,000 to $124,999","Estimate; Householder 25 to 44 years: - $125,000 to $149,999","Estimate; Householder 25 to 44 years: - $150,000 to $199,999","Estimate; Householder 25 to 44 years: - $200,000 or more",Estimate; Householder 45 to 64 years:,"Estimate; Householder 45 to 64 years: - Less than $10,000","Estimate; Householder 45 to 64 years: - $10,000 to $14,999","Estimate; Householder 45 to 64 years: - $15,000 to $19,999","Estimate; Householder 45 to 64 years: - $20,000 to $24,999","Estimate; Householder 45 to 64 years: - $25,000 to $29,999","Estimate; Householder 45 to 64 years: - $30,000 to $34,999","Estimate; Householder 45 to 64 years: - $35,000 to $39,999","Estimate; Householder 45 to 64 years: - $40,000 to $44,999","Estimate; Householder 45 to 64 years: - $45,000 to $49,999","Estimate; Householder 45 to 64 years: - $50,000 to $59,999","Estimate; Householder 45 to 64 years: - $60,000 to $74,999","Estimate; Householder 45 to 64 years: - $75,000 to $99,999","Estimate; Householder 45 to 64 years: - $100,000 to $124,999","Estimate; Householder 45 to 64 years: - $125,000 to $149,999","Estimate; Householder 45 to 64 years: - $150,000 to $199,999","Estimate; Householder 45 to 64 years: - $200,000 or more",Estimate; Householder 65 years and over:,"Estimate; Householder 65 years and over: - Less than $10,000","Estimate; Householder 65 years and over: - $10,000 to $14,999","Estimate; Householder 65 years and over: - $15,000 to $19,999","Estimate; Householder 65 years and over: - $20,000 to $24,999","Estimate; Householder 65 years and over: - $25,000 to $29,999","Estimate; Householder 65 years and over: - $30,000 to $34,999","Estimate; Householder 65 years and over: - $35,000 to $39,999","Estimate; Householder 65 years and over: - $40,000 to $44,999","Estimate; Householder 65 years and over: - $45,000 to $49,999","Estimate; Householder 65 years and over: - $50,000 to $59,999","Estimate; Householder 65 years and over: - $60,000 to $74,999","Estimate; Householder 65 years and over: - $75,000 to $99,999","Estimate; Householder 65 years and over: - $100,000 to $124,999","Estimate; Householder 65 years and over: - $125,000 

In [10]:
dat_trim.shape

(33120, 70)

We have some columns counting the totals overall and by age group. We'll want to drop these as we're going to be agging the data granularly, so having the totals included will mess up the way we'll eventually be summing the rows. First we identify these "total: columns:

In [11]:
dat_trim.iloc[:, np.r_[1:3, 19:20, 36:37, 53:54]].head()

,Estimate; Total:,Estimate; Householder under 25 years:,Estimate; Householder 25 to 44 years:,Estimate; Householder 45 to 64 years:,Estimate; Householder 65 years and over:
1,5818,23,1513,2540,1742
2,12719,99,2775,5908,3937
3,19009,513,5229,6885,6382
4,1959,9,418,855,677
5,9120,138,2142,3396,3444


Before we drop the totals, let's first sum the total columns so we know how many households we had in the original, untouched data. Then as we build our new data set we can use this as a benchmark to verify that we aren't accidentally dropping useful data.

In [12]:
dat_trim["Estimate; Total:"] = dat_trim["Estimate; Total:"].astype('int64', copy=False)
dat_trim["Estimate; Total:"].sum()

120046688

We'll drop these columns from the data set we're going to be building, but we'll keep these totals so we can sanity check ourselves when we've built a melted, long version of this wide data set.

In [13]:
totals = [1,2,19,36,53]
dat_trim.drop(dat_trim.columns[totals],axis=1,inplace=True)
dat_trim.head()

,Id2,"Estimate; Householder under 25 years: - Less than $10,000","Estimate; Householder under 25 years: - $10,000 to $14,999","Estimate; Householder under 25 years: - $15,000 to $19,999","Estimate; Householder under 25 years: - $20,000 to $24,999","Estimate; Householder under 25 years: - $25,000 to $29,999","Estimate; Householder under 25 years: - $30,000 to $34,999","Estimate; Householder under 25 years: - $35,000 to $39,999","Estimate; Householder under 25 years: - $40,000 to $44,999","Estimate; Householder under 25 years: - $45,000 to $49,999","Estimate; Householder under 25 years: - $50,000 to $59,999","Estimate; Householder under 25 years: - $60,000 to $74,999","Estimate; Householder under 25 years: - $75,000 to $99,999","Estimate; Householder under 25 years: - $100,000 to $124,999","Estimate; Householder under 25 years: - $125,000 to $149,999","Estimate; Householder under 25 years: - $150,000 to $199,999","Estimate; Householder under 25 years: - $200,000 or more","Estimate; Householder 25 to 44 years: - Less than $10,000","Estimate; Householder 25 to 44 years: - $10,000 to $14,999","Estimate; Householder 25 to 44 years: - $15,000 to $19,999","Estimate; Householder 25 to 44 years: - $20,000 to $24,999","Estimate; Householder 25 to 44 years: - $25,000 to $29,999","Estimate; Householder 25 to 44 years: - $30,000 to $34,999","Estimate; Householder 25 to 44 years: - $35,000 to $39,999","Estimate; Householder 25 to 44 years: - $40,000 to $44,999","Estimate; Householder 25 to 44 years: - $45,000 to $49,999","Estimate; Householder 25 to 44 years: - $50,000 to $59,999","Estimate; Householder 25 to 44 years: - $60,000 to $74,999","Estimate; Householder 25 to 44 years: - $75,000 to $99,999","Estimate; Householder 25 to 44 years: - $100,000 to $124,999","Estimate; Householder 25 to 44 years: - $125,000 to $149,999","Estimate; Householder 25 to 44 years: - $150,000 to $199,999","Estimate; Householder 25 to 44 years: - $200,000 or more","Estimate; Householder 45 to 64 years: - Less than $10,000","Estimate; Householder 45 to 64 years: - $10,000 to $14,999","Estimate; Householder 45 to 64 years: - $15,000 to $19,999","Estimate; Householder 45 to 64 years: - $20,000 to $24,999","Estimate; Householder 45 to 64 years: - $25,000 to $29,999","Estimate; Householder 45 to 64 years: - $30,000 to $34,999","Estimate; Householder 45 to 64 years: - $35,000 to $39,999","Estimate; Householder 45 to 64 years: - $40,000 to $44,999","Estimate; Householder 45 to 64 years: - $45,000 to $49,999","Estimate; Householder 45 to 64 years: - $50,000 to $59,999","Estimate; Householder 45 to 64 years: - $60,000 to $74,999","Estimate; Householder 45 to 64 years: - $75,000 to $99,999","Estimate; Householder 45 to 64 years: - $100,000 to $124,999","Estimate; Householder 45 to 64 years: - $125,000 to $149,999","Estimate; Householder 45 to 64 years: - $150,000 to $199,999","Estimate; Householder 45 to 64 years: - $200,000 or more","Estimate; Householder 65 years and over: - Less than $10,000","Estimate; Householder 65 years and over: - $10,000 to $14,999","Estimate; Householder 65 years and over: - $15,000 to $19,999","Estimate; Householder 65 years and over: - $20,000 to $24,999","Estimate; Householder 65 years and over: - $25,000 to $29,999","Estimate; Householder 65 years and over: - $30,000 to $34,999","Estimate; Householder 65 years and over: - $35,000 to $39,999","Estimate; Householder 65 years and over: - $40,000 to $44,999","Estimate; Householder 65 years and over: - $45,000 to $49,999","Estimate; Householder 65 years and over: - $50,000 to $59,999","Estimate; Householder 65 years and over: - $60,000 to $74,999","Estimate; Householder 65 years and over: - $75,000 to $99,999","Estimate; Householder 65 years and over: - $100,000 to $124,999","Estimate; Householder 65 years and over: - $125,000 to $149,999","Estimate; Householder 65 years and over: - $150,000 to $199,999","Estimate; Householder 65 years and over: - $200,000 or more"
1,601,8,0,15,0,0,0,0,0,0,0,0,0,

In [14]:
dat_trim.shape

(33120, 65)

We want to flip this data set from wide (65 columns) to long (just a column for age, income, and count).

In [15]:
# let the zip code be the id, and all the income buckets will get flattened into one column
# we'll run the function once for each age bucket, where an age bucket spans columns a to b
def melt_data(dat, a, b, age_bracket):
    new_dat = dat.iloc[:, np.r_[0:1, a:b]]
    stack = pd.melt(new_dat, id_vars=['Id2'], value_vars=list(new_dat)[1:], var_name='income_bucket', value_name='count')
    stack['age_bracket'] = age_bracket
    return stack

In [16]:
dat_trim.head()

,Id2,"Estimate; Householder under 25 years: - Less than $10,000","Estimate; Householder under 25 years: - $10,000 to $14,999","Estimate; Householder under 25 years: - $15,000 to $19,999","Estimate; Householder under 25 years: - $20,000 to $24,999","Estimate; Householder under 25 years: - $25,000 to $29,999","Estimate; Householder under 25 years: - $30,000 to $34,999","Estimate; Householder under 25 years: - $35,000 to $39,999","Estimate; Householder under 25 years: - $40,000 to $44,999","Estimate; Householder under 25 years: - $45,000 to $49,999","Estimate; Householder under 25 years: - $50,000 to $59,999","Estimate; Householder under 25 years: - $60,000 to $74,999","Estimate; Householder under 25 years: - $75,000 to $99,999","Estimate; Householder under 25 years: - $100,000 to $124,999","Estimate; Householder under 25 years: - $125,000 to $149,999","Estimate; Householder under 25 years: - $150,000 to $199,999","Estimate; Householder under 25 years: - $200,000 or more","Estimate; Householder 25 to 44 years: - Less than $10,000","Estimate; Householder 25 to 44 years: - $10,000 to $14,999","Estimate; Householder 25 to 44 years: - $15,000 to $19,999","Estimate; Householder 25 to 44 years: - $20,000 to $24,999","Estimate; Householder 25 to 44 years: - $25,000 to $29,999","Estimate; Householder 25 to 44 years: - $30,000 to $34,999","Estimate; Householder 25 to 44 years: - $35,000 to $39,999","Estimate; Householder 25 to 44 years: - $40,000 to $44,999","Estimate; Householder 25 to 44 years: - $45,000 to $49,999","Estimate; Householder 25 to 44 years: - $50,000 to $59,999","Estimate; Householder 25 to 44 years: - $60,000 to $74,999","Estimate; Householder 25 to 44 years: - $75,000 to $99,999","Estimate; Householder 25 to 44 years: - $100,000 to $124,999","Estimate; Householder 25 to 44 years: - $125,000 to $149,999","Estimate; Householder 25 to 44 years: - $150,000 to $199,999","Estimate; Householder 25 to 44 years: - $200,000 or more","Estimate; Householder 45 to 64 years: - Less than $10,000","Estimate; Householder 45 to 64 years: - $10,000 to $14,999","Estimate; Householder 45 to 64 years: - $15,000 to $19,999","Estimate; Householder 45 to 64 years: - $20,000 to $24,999","Estimate; Householder 45 to 64 years: - $25,000 to $29,999","Estimate; Householder 45 to 64 years: - $30,000 to $34,999","Estimate; Householder 45 to 64 years: - $35,000 to $39,999","Estimate; Householder 45 to 64 years: - $40,000 to $44,999","Estimate; Householder 45 to 64 years: - $45,000 to $49,999","Estimate; Householder 45 to 64 years: - $50,000 to $59,999","Estimate; Householder 45 to 64 years: - $60,000 to $74,999","Estimate; Householder 45 to 64 years: - $75,000 to $99,999","Estimate; Householder 45 to 64 years: - $100,000 to $124,999","Estimate; Householder 45 to 64 years: - $125,000 to $149,999","Estimate; Householder 45 to 64 years: - $150,000 to $199,999","Estimate; Householder 45 to 64 years: - $200,000 or more","Estimate; Householder 65 years and over: - Less than $10,000","Estimate; Householder 65 years and over: - $10,000 to $14,999","Estimate; Householder 65 years and over: - $15,000 to $19,999","Estimate; Householder 65 years and over: - $20,000 to $24,999","Estimate; Householder 65 years and over: - $25,000 to $29,999","Estimate; Householder 65 years and over: - $30,000 to $34,999","Estimate; Householder 65 years and over: - $35,000 to $39,999","Estimate; Householder 65 years and over: - $40,000 to $44,999","Estimate; Householder 65 years and over: - $45,000 to $49,999","Estimate; Householder 65 years and over: - $50,000 to $59,999","Estimate; Householder 65 years and over: - $60,000 to $74,999","Estimate; Householder 65 years and over: - $75,000 to $99,999","Estimate; Householder 65 years and over: - $100,000 to $124,999","Estimate; Householder 65 years and over: - $125,000 to $149,999","Estimate; Householder 65 years and over: - $150,000 to $199,999","Estimate; Householder 65 years and over: - $200,000 or more"
1,601,8,0,15,0,0,0,0,0,0,0,0,0,

In [17]:
under_25 = melt_data(dat_trim, 1, 17, 'under 25')
under_45 = melt_data(dat_trim, 17, 33, '25 to 44')
under_65 = melt_data(dat_trim, 33, 49, '45 to 65')
over_65 = melt_data(dat_trim, 49, 65, '65 +')


In [18]:
bind = pd.concat([under_25, under_45, under_65, over_65])
bind.head()

,Id2,income_bucket,count,age_bracket
0,601,Estimate; Householder under 25 years: - Less t...,8,under 25
1,602,Estimate; Householder under 25 years: - Less t...,71,under 25
2,603,Estimate; Householder under 25 years: - Less t...,354,under 25
3,606,Estimate; Householder under 25 years: - Less t...,9,under 25
4,610,Estimate; Householder under 25 years: - Less t...,60,under 25


The income bucket rows are wordy, we can trim everything preceding the dash to make the data more digestible.

In [19]:
bind['income_bucket'] = bind['income_bucket'].apply(lambda x: x.split('- ',1)[1])
bind.head()

,Id2,income_bucket,count,age_bracket
0,601,"Less than $10,000",8,under 25
1,602,"Less than $10,000",71,under 25
2,603,"Less than $10,000",354,under 25
3,606,"Less than $10,000",9,under 25
4,610,"Less than $10,000",60,under 25


Next we verify that we still have all of our households.

In [20]:
bind['count'] = bind['count'].astype('int64', copy=False)
bind['count'].sum()

120046688

As a sanity check, we see this total sums to the same number we got when summing the "Estimate Total" column from the original data set. 
This confirms that we built our long data set correctly so we can move forward.

As a sanity check, we see there are roughly 120 million households in our new, long dataset. Now, we can filter the data by age bracket, income bucket, and zip code. The next steps will be to filter these households to ones that contain women by household type (married, single mom, single woman), add a finer level of granularity along the age facet, and finally we will join in a zip code data set to gather additional geographical and regional information (state, city, region, division etc).

### Part 2: Merging and Multiplying

The US Census provides additonal data on household types. Family vs non-family. Family house holds can be either heterosexual married couples are single parents with children. Non family households can be homosexual couples without kids or single men/women. The data sets can be found [here](https://www.census.gov/data/tables/time-series/demo/income-poverty/cps-hinc/hinc-04.html).
This data is taken from 2017 and cites 127 million households. The zip code data set we loaded from the american fact finder cited 120 million households. The discrepancy is reasonable given that the american fact finder uses a five year estimate (thus averages the past 5 years to get a rolling 5 year average for each year). We will be using the 2017 census data as a means for understanding the percentage distribution of households that contain women, so the discrepancy in counts shouldn't affect the end result. The methodology used was to gather 5 separate data sets: married-couple family households, female house holder (no spouse), male house holder (no spouse), nonfamily female householder, and nonfamily male householder.
Then, of the total count of households within each bucket we assumed that only married households or female headed-households contain. The result gleaned from slicing the data this way is that 12.4% of households are either single dads with children or male householders with no spouse present. If we multiply all our household counts by .876, then we retain the count of households which contain women.

In [21]:
df1 = pd.read_pickle('overview')
df1

,household_type,grand_total,narrow_count_x,pct
0,male_alone,127562000.0,16009000.0,0.125500
1,male_not_alone,127562000.0,4996000.0,0.039165
2,married,127562000.0,61242000.0,0.480096
3,single_dad,127562000.0,6425000.0,0.050368
4,single_mom,127562000.0,15422000.0,0.120898
5,single_woman,127562000.0,23468000.0,0.183973


##### Estimate counts by household type based on overview percentages

In [22]:
bind[["count"]] = bind[["count"]].astype(float)
bind['married'] = bind['count'] * 0.480096
bind['male_not_alone']= bind['count']*0.039165
bind['male_alone'] = bind['count']*.1255
bind['single_dad'] = bind['count']*0.050368
bind['single_mom'] = bind['count']*0.120898
bind['single_woman'] = bind['count']*0.183973

est_households = pd.melt(bind, id_vars=['Id2', 'income_bucket', 'age_bracket', 'count'], value_vars=['married', 'male_alone', 'male_not_alone', 'single_dad', 'single_mom', 'single_woman'], var_name = 'household_type', value_name = 'house_count')

In [23]:
est_households.query('age_bracket == "under 25" and income_bucket == "Less than $10,000" and Id2 == "601"').sum()

Id2                                              601601601601601601
income_bucket     Less than $10,000Less than $10,000Less than $1...
age_bracket        under 25under 25under 25under 25under 25under 25
count                                                            48
household_type    marriedmale_alonemale_not_alonesingle_dadsingl...
house_count                                                       8
dtype: object

In [24]:
est_households.query('age_bracket == "under 25" and income_bucket == "Less than $10,000" and Id2 == "601"')

,Id2,income_bucket,age_bracket,count,household_type,house_count
0,601,"Less than $10,000",under 25,8.0,married,3.840768
2119680,601,"Less than $10,000",under 25,8.0,male_alone,1.004000
4239360,601,"Less than $10,000",under 25,8.0,male_not_alone,0.313320
6359040,601,"Less than $10,000",under 25,8.0,single_dad,0.402944
8478720,601,"Less than $10,000",under 25,8.0,single_mom,0.967184
10598400,601,"Less than $10,000",under 25,8.0,single_woman,1.471784


In [25]:
est_households.house_count.sum()

120046687.99999993

In [26]:
est_households.shape

(12718080, 6)

In [27]:
pct_map = pd.read_pickle('pct_map')
pct_map.head()

,age_bracket,household_type,narrow_age_bucket,wide_income,narrow_count_x,narrow_count_y,pct
0,25 to 44,male_alone,25 to 29 years,"$100,000 to $124,999",62000.0,294000.0,0.210884
1,25 to 44,male_alone,30 to 34 years,"$100,000 to $124,999",71000.0,294000.0,0.241497
2,25 to 44,male_alone,35 to 39 years,"$100,000 to $124,999",97000.0,294000.0,0.329932
3,25 to 44,male_alone,40 to 44 years,"$100,000 to $124,999",64000.0,294000.0,0.217687
4,25 to 44,male_alone,25 to 29 years,"$125,000 to $149,999",38000.0,144000.0,0.263889


In [28]:
pct_map.wide_income.unique()

array(['$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more', '$75,000 to $99,999',
       'Under 75k'], dtype=object)

In [29]:
est_households.income_bucket.unique()

array(['Less than $10,000', '$10,000 to $14,999', '$15,000 to $19,999',
       '$20,000 to $24,999', '$25,000 to $29,999', '$30,000 to $34,999',
       '$35,000 to $39,999', '$40,000 to $44,999', '$45,000 to $49,999',
       '$50,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999',
       '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more'], dtype=object)

In [30]:
def classify_income2(str_range):
    
    under_75k = ['Under $5,000', '$5,000 to $9,999', '$10,000 to $14,999',
       '$15,000 to $19,999', '$20,000 to $24,999', '$25,000 to $29,999',
       '$30,000 to $34,999', '$35,000 to $39,999', '$40,000 to $44,999',
       '$45,000 to $49,999', '$50,000 to $54,999', '$55,000 to $59,999',
       '$60,000 to $64,999', '$65,000 to $69,999', '$70,000 to $74,999']
    
    tier2 = ['$75,000 to $99,999']
    
    tier3 = ['$100,000 to $124,999']
       
    tier4 = ['$125,000 to $149,999']
    
    tier5 = ['$150,000 to $199,999']
    
    tier6 = ['$200,000 and over']
    
    if str_range in under_75k:
        return 'Under 75k'
    
    elif str_range in tier2:
        return '$75,000 to $99,999'
    
    elif str_range in tier3:
        return '$100,000 to $124,999'
    
    elif str_range in tier4:
        return '$125,000 to $149,999'
    
    elif str_range in tier5:
        return '$150,000 to $199,999'
    else:
        return '$200,000 or more'
    

In [31]:
est_households['wide_income'] = est_households['income_bucket'].apply(classify_income2)
est_households.wide_income.unique()

array(['$200,000 or more', 'Under 75k', '$75,000 to $99,999',
       '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999'], dtype=object)

In [32]:
est_households.query('household_type == "single_woman" and age_bracket == "25 to 44" and wide_income == "$200,000 or more" and Id2 == "601"')

,Id2,income_bucket,age_bracket,count,household_type,house_count,wide_income
11128320,601,"Less than $10,000",25 to 44,727.0,single_woman,133.748371,"$200,000 or more"
11426400,601,"$50,000 to $59,999",25 to 44,66.0,single_woman,12.142218,"$200,000 or more"
11459520,601,"$60,000 to $74,999",25 to 44,49.0,single_woman,9.014677,"$200,000 or more"
11625120,601,"$200,000 or more",25 to 44,0.0,single_woman,0.000000,"$200,000 or more"


In [33]:
est_households.house_count.sum()

120046687.99999993

In [34]:
pct_map.query('age_bracket == "25 to 44"').head()

,age_bracket,household_type,narrow_age_bucket,wide_income,narrow_count_x,narrow_count_y,pct
0,25 to 44,male_alone,25 to 29 years,"$100,000 to $124,999",62000.0,294000.0,0.210884
1,25 to 44,male_alone,30 to 34 years,"$100,000 to $124,999",71000.0,294000.0,0.241497
2,25 to 44,male_alone,35 to 39 years,"$100,000 to $124,999",97000.0,294000.0,0.329932
3,25 to 44,male_alone,40 to 44 years,"$100,000 to $124,999",64000.0,294000.0,0.217687
4,25 to 44,male_alone,25 to 29 years,"$125,000 to $149,999",38000.0,144000.0,0.263889


In [35]:
pct_map.pct.sum()

144.0

In [36]:
est_house = est_households[['Id2', 'wide_income', 'age_bracket', 'house_count', 'household_type']]
est_house.house_count.sum()

120046687.99999993

In [37]:
est_h = est_house.groupby(['wide_income', 'Id2', 'age_bracket', 'household_type']).agg('sum').reset_index()
est_h.house_count.sum()

120046687.99999999

In [38]:
est_h.shape

(4769280, 5)

In [39]:
est_h.head()

,wide_income,Id2,age_bracket,household_type,house_count
0,"$100,000 to $124,999",10001,25 to 44,male_alone,76.680500
1,"$100,000 to $124,999",10001,25 to 44,male_not_alone,23.929815
2,"$100,000 to $124,999",10001,25 to 44,married,293.338656
3,"$100,000 to $124,999",10001,25 to 44,single_dad,30.774848
4,"$100,000 to $124,999",10001,25 to 44,single_mom,73.868678


In [40]:
pct_map.shape

(432, 7)

In [41]:
pct_map.head()

,age_bracket,household_type,narrow_age_bucket,wide_income,narrow_count_x,narrow_count_y,pct
0,25 to 44,male_alone,25 to 29 years,"$100,000 to $124,999",62000.0,294000.0,0.210884
1,25 to 44,male_alone,30 to 34 years,"$100,000 to $124,999",71000.0,294000.0,0.241497
2,25 to 44,male_alone,35 to 39 years,"$100,000 to $124,999",97000.0,294000.0,0.329932
3,25 to 44,male_alone,40 to 44 years,"$100,000 to $124,999",64000.0,294000.0,0.217687
4,25 to 44,male_alone,25 to 29 years,"$125,000 to $149,999",38000.0,144000.0,0.263889


In [42]:
pcts = pct_map[['age_bracket', 'household_type', 'narrow_age_bucket', 'wide_income', 'pct']]
finalement = pd.merge(est_h, pcts, on = ['age_bracket', 'wide_income', 'household_type'], how = 'inner')

In [43]:
finalement.shape

(14307840, 7)

In [44]:
finalement.head()

,wide_income,Id2,age_bracket,household_type,house_count,narrow_age_bucket,pct
0,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,25 to 29 years,0.210884
1,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,30 to 34 years,0.241497
2,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,35 to 39 years,0.329932
3,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,40 to 44 years,0.217687
4,"$100,000 to $124,999",10002,25 to 44,male_alone,107.9300,25 to 29 years,0.210884


In [45]:
finalement.house_count.sum()

436974066.99999976

In [46]:
f1 = finalement.copy()
f1['slice_count'] = f1.house_count * f1.pct

In [47]:
f1.slice_count.sum()

120046688.00000013

In [48]:
f1.shape

(14307840, 8)

In [49]:
f1[['slice_count', 'household_type']].groupby('household_type').agg('sum')

,slice_count
household_type,
male_alone,1.506586e+07
male_not_alone,4.701629e+06
married,5.763393e+07
single_dad,6.046512e+06
single_mom,1.451340e+07
single_woman,2.208535e+07


In [50]:
est_house[['house_count', 'household_type']].groupby('household_type').agg('sum')

,house_count
household_type,
male_alone,1.506586e+07
male_not_alone,4.701629e+06
married,5.763393e+07
single_dad,6.046512e+06
single_mom,1.451340e+07
single_woman,2.208535e+07


In [51]:
f1.household_type.unique()

array(['male_alone', 'male_not_alone', 'married', 'single_dad',
       'single_mom', 'single_woman'], dtype=object)

In [52]:
f1.wide_income.unique()

array(['$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more', '$75,000 to $99,999',
       'Under 75k'], dtype=object)

In [53]:
f1.query('wide_income != "Under 75k"').slice_count.sum()

75242427.00000016

In [54]:
f1.query('wide_income != "Under 75k" and age_bracket != "under 25" and age_bracket != "25 to 44"').slice_count.sum()

47228204.99999997

In [55]:
f1.query('wide_income != "Under 75k" and age_bracket != "under 25" and age_bracket != "25 to 44" and household_type != "single_dad"').slice_count.sum()

44849414.77056004

In [56]:
f1.query('wide_income != "Under 75k" and age_bracket != "under 25" and age_bracket != "25 to 44" and household_type != "single_dad" and household_type != "male_alone" and household_type != "male_not_alone"').slice_count.sum()

37072582.39423502

Next we'll join in some additional zip code data to gain further information from these zip codes (state, region, division, city etc)

In [62]:
uszips = pd.read_csv('uszips.csv')
uszips.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,601,18.1800,-66.7522,Adjuntas,PR,Puerto Rico,True,NaN,18570,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico
1,602,18.3607,-67.1752,Aguada,PR,Puerto Rico,True,NaN,41520,523.7,72003,Aguada,{'72003':100},False,False,America/Puerto_Rico
2,603,18.4544,-67.1220,Aguadilla,PR,Puerto Rico,True,NaN,54689,667.9,72005,Aguadilla,{'72005':100},False,False,America/Puerto_Rico
3,606,18.1672,-66.9383,Maricao,PR,Puerto Rico,True,NaN,6615,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",False,False,America/Puerto_Rico
4,610,18.2903,-67.1224,Anasco,PR,Puerto Rico,True,NaN,29016,311.9,72011,Añasco,"{'72003':0.55,'72011':99.45}",False,False,America/Puerto_Rico


In [63]:
len(list(uszips.query('state_id != "PR"').zip.unique()))

32968

In [64]:
len(list(final_data_set.zip_code.unique()))

NameError: name 'final_data_set' is not defined

In [65]:
final_data_set.shape

NameError: name 'final_data_set' is not defined

In [66]:
uszips.shape

(33099, 16)

In [67]:
f1.dtypes

wide_income           object
Id2                   object
age_bracket           object
household_type        object
house_count          float64
narrow_age_bucket     object
pct                  float64
slice_count          float64
dtype: object

In [68]:
f1['Id2']  = f1['Id2'].astype(str).astype(int)

In [69]:
dat = pd.merge(f1, uszips, left_on = 'Id2', right_on = 'zip', how = 'left')

In [70]:
dat.head()

,wide_income,Id2,age_bracket,household_type,house_count,narrow_age_bucket,pct,slice_count,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,25 to 29 years,0.210884,16.170718,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York
1,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,30 to 34 years,0.241497,18.518080,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York
2,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,35 to 39 years,0.329932,25.299349,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York
3,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,40 to 44 years,0.217687,16.692354,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York
4,"$100,000 to $124,999",10002,25 to 44,male_alone,107.9300,25 to 29 years,0.210884,22.760748,10002.0,40.7157,-73.9863,New York,NY,New York,True,NaN,81410.0,35769.1,36061.0,New York,{'36061':100},False,False,America/New_York


In [71]:
len(list(dat.state_id.unique()))

53

In [72]:
regions = pd.read_csv('regions.csv')
regions.head()

,state,region,division,state_abr
0,Connecticut,Northeast,New England,CT
1,Maine,Northeast,New England,ME
2,Massachusetts,Northeast,New England,MA
3,New Hampshire,Northeast,New England,NH
4,Rhode Island,Northeast,New England,RI


In [73]:
dat['state_id']  = dat['state_id'].astype(str)
regions['state_abr']  = regions['state_abr'].astype(str)
dat_r = pd.merge(dat, regions, left_on = 'state_id', right_on = 'state_abr', how = 'left')
dat_r.head()

,wide_income,Id2,age_bracket,household_type,house_count,narrow_age_bucket,pct,slice_count,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone,state,region,division,state_abr
0,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,25 to 29 years,0.210884,16.170718,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York,New York,Northeast,Middle Atlantic,NY
1,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,30 to 34 years,0.241497,18.518080,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York,New York,Northeast,Middle Atlantic,NY
2,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,35 to 39 years,0.329932,25.299349,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York,New York,Northeast,Middle Atlantic,NY
3,"$100,000 to $124,999",10001,25 to 44,male_alone,76.6805,40 to 44 years,0.217687,16.692354,10001.0,40.7506,-73.9971,New York,NY,New York,True,NaN,21102.0,13254.0,36061.0,New York,{'36061':100},False,False,America/New_York,New York,Northeast,Middle Atlantic,NY
4,"$100,000 to $124,999",10002,25 to 44,male_alone,107.9300,25 to 29 years,0.210884,22.760748,10002.0,40.7157,-73.9863,New York,NY,New York,True,NaN,81410.0,35769.1,36061.0,New York,{'36061':100},False,False,America/New_York,New York,Northeast,Middle Atlantic,NY


In [74]:
dat_r.tail()

,wide_income,Id2,age_bracket,household_type,house_count,narrow_age_bucket,pct,slice_count,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone,state,region,division,state_abr
14307835,Under 75k,99923,under 25,single_woman,0.000000,15 to 24 years,1.0,0.000000,99923.0,55.9811,-130.0405,Hyder,AK,Alaska,True,NaN,87.0,2.4,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka,Alaska,West,Pacific,AK
14307836,Under 75k,99925,under 25,single_woman,1.103838,15 to 24 years,1.0,1.103838,99925.0,55.5577,-132.9763,Klawock,AK,Alaska,True,NaN,819.0,5.7,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka,Alaska,West,Pacific,AK
14307837,Under 75k,99926,under 25,single_woman,1.287811,15 to 24 years,1.0,1.287811,99926.0,55.1262,-131.4893,Metlakatla,AK,Alaska,True,NaN,1460.0,4.2,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Metlakatla,Alaska,West,Pacific,AK
14307838,Under 75k,99927,under 25,single_woman,0.000000,15 to 24 years,1.0,0.000000,99927.0,56.2510,-133.3757,Point Baker,AK,Alaska,True,NaN,94.0,0.2,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka,Alaska,West,Pacific,AK
14307839,Under 75k,99929,under 25,single_woman,2.575622,15 to 24 years,1.0,2.575622,99929.0,56.3670,-131.9329,Wrangell,AK,Alaska,True,NaN,2338.0,0.4,2275.0,Wrangell,{'02275':100},False,False,America/Sitka,Alaska,West,Pacific,AK


In [80]:
dat_r.household_type.unique()

array(['male_alone', 'male_not_alone', 'married', 'single_dad',
       'single_mom', 'single_woman'], dtype=object)

In [69]:
dat_r.shape

(14307840, 28)

In [70]:
dat_r.slice_count.sum()

120046688.00000013

In [71]:
dat_r.query('state_id != "PR"').slice_count.sum()

118824571.00000006

In [168]:
dat_r.to_csv("full_market_analysis.csv",index=False)

In [169]:
dat_r[['household_type', 'slice_count']].groupby('household_type').agg('sum')

,slice_count
household_type,
male_alone,1.506586e+07
male_not_alone,4.701629e+06
married,5.763393e+07
single_dad,6.046512e+06
single_mom,1.451340e+07
single_woman,2.208535e+07
